In [1]:
import yaml

# Import directory setting and volumes to mount from machine.yaml
with open(r'machine.yaml') as file:
    machinecfg = yaml.load(file, Loader=yaml.FullLoader)

# Transform volumes dict into "host=bind" strings 
volume_list = []
for key, value in machinecfg['volumes'].items():
    volume_list += [f'{value}:/{key}']

# Import parameters needed to render templates
with open(r'diagnostic.yaml') as file:
    diagcfg = yaml.load(file, Loader=yaml.FullLoader)



In [2]:
from jinja2 import Template, FileSystemLoader, Environment

templateLoader = FileSystemLoader(searchpath="./templates")
templateEnv = Environment(loader=templateLoader)

# Now use jinja2 to render al top-level keys in recipe file
# as templates into actual recipes, config files etc.

for k in diagcfg:
    dest_file = diagcfg[k]['filename']
    template_file = diagcfg[k]['template']
    template = templateEnv.get_template(template_file)
    template.stream(diagcfg[k]['parameters']).dump(dest_file)
    

In [3]:
# Run actual diagnostic docker

import docker
client = docker.from_env()
output = client.containers.run(image=machinecfg['image'],
                      command=machinecfg['command'], 
                      volumes=volume_list, 
                      environment=machinecfg['environment'])


In [23]:
# Postprocessing TBD